# 4. Review generated data similarity(Regression)

## 라이브러리 불러오기

In [1]:
import pandas as pd
import copy
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
import copy
import time

from collections import defaultdict
from itertools import islice, combinations
from datetime import datetime as dt

import warnings
warnings.filterwarnings("ignore")

In [3]:
#Jupiter Cell Full Screen View
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# useful for debuging (print the results of both formulas and functions entered in one cell of Jupiter)
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"
# Each column width at maximum (print all column contents)
pd.set_option('display.max_colwidth', -1)
# Show up to 500 rows
pd.set_option('display.max_rows', 500)
# Display up to 500 columns
pd.set_option('display.max_columns', 500)
# Total length of data frame
pd.set_option('display.width', 1000)

print('ready to run')
# logging starttime 
startTime = time.time()

ready to run


## 데이터 불러오기 / 데이터 전처리하기

- 범주형 데이터 칼럼
- 숫자형 데이터 칼럼

In [4]:
category_cols = ['dec_num','dec_date','dec_custom_code','imp_dec_code','imp_trd_code','imp_typ_code',\
                 'collect_code','typ_transport_code','dec_mark','importer','ovs_cust_code',\
                 'exps_carr_code','HS10','country_ship_code','country_orig_code','trff_class_code',\
                 'country_orig_mark_code','crime_yn','key_exposure']

In [5]:
number_cols = ['trff_rate','dec_weight','taxabal_price_KRW']

### 원본 데이터

In [6]:
df_base = pd.read_csv('df_syn_en.csv', encoding='utf-8-sig') #새로운 가상데이터

In [7]:
copy_base = copy.deepcopy(df_base)

In [8]:
copy_base_category = copy_base[category_cols]
copy_base_number = copy_base[number_cols]

#### 범주형 데이터 라벨링

In [9]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt 

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

for column_name,item in copy_base_category.iteritems(): 
    encoder.fit(item)
    labels = encoder.transform(item)
    copy_base_category[column_name] = labels

#### 숫자형 데이터 최소-최대 정규화

In [10]:
def normalize(column):
    return (column - column.min())/(column.max() - column.min())

In [11]:
copy_base_number_nom = copy_base_number.apply(normalize)

#### 데이터 결합

In [12]:
copy_base_total = pd.concat([copy_base_category,copy_base_number_nom], axis=1)

#### 훈련데이터, 테스트 데이터 분리

In [13]:
y = copy_base_total['crime_yn']
X = copy_base_total.drop(columns=['crime_yn','key_exposure'])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 합성 데이터

In [15]:
df_syn = pd.read_csv('./data_sample/df_syn_en_14.csv', encoding='utf-8-sig') #새로운 가상데이터

In [16]:
copy_syn = copy.deepcopy(df_syn)

In [17]:
copy_syn_category = copy_syn[category_cols]
copy_syn_number = copy_syn[number_cols]

#### 범주형 데이터 라벨링

In [18]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt 

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

for column_name,item in copy_syn_category.iteritems(): 
    encoder.fit(item)
    labels = encoder.transform(item)
    copy_syn_category[column_name] = labels

#### 숫자형 데이터 최소-최대 정규화

In [19]:
def normalize(column):
    return (column - column.min())/(column.max() - column.min())

In [20]:
copy_syn_number_nom = copy_syn_number.apply(normalize)

#### 데이터 결합

In [21]:
copy_syn_total = pd.concat([copy_syn_category,copy_syn_number_nom], axis=1)

#### 훈련데이터, 테스트 데이터 분리

In [22]:
y_syn = copy_syn_total['crime_yn']
X_syn = copy_syn_total.drop(columns=['crime_yn','key_exposure'])

In [23]:
X_syn_train, X_syn_test, y_syn_train, y_syn_test = train_test_split(X_syn, y_syn, test_size=0.2, random_state=42)

## 분석하기

### 선형 회귀분석

In [24]:
from sklearn.linear_model import LinearRegression

#### 원본데이터 (X_train, X_test, y_train, y_test)

- 원본데이터 회귀모델 학습 

In [25]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [26]:
score = lr.score(X_test, y_test)
print("Model score:", score)

Model score: 0.00973790576489375


- 원본데이터 예측값

In [27]:
y_pred = lr.predict(X_test)

In [28]:
index = X_test['dec_num'].index.tolist()

In [29]:
df = pd.DataFrame({'index' : index ,'pred': y_pred})

- 원본데이터 우범 회귀 예측값 상위 5%의 우범률

In [30]:
top_5percent = df['pred'].quantile(0.95)
result = df[df['pred'] > top_5percent]

In [31]:
y_test_top5 = y_test[y_test.index.isin(result['index'])]

In [32]:
y_test_top5_df = pd.DataFrame({'crime_yn' :y_test_top5})

In [33]:
base_5 = y_test_top5_df[y_test_top5_df['crime_yn']==1].count() / y_test_top5_df.count()

- 원본데이터 우범 회귀 예측값 상위 10%의 우범률

In [34]:
top_10percent = df['pred'].quantile(0.90)
result = df[df['pred'] > top_10percent]

In [35]:
y_test_top10 = y_test[y_test.index.isin(result['index'])]

In [36]:
y_test_top10_df = pd.DataFrame({'crime_yn' : y_test_top10})

In [37]:
base_10= y_test_top10_df[y_test_top10_df['crime_yn']==1].count() / y_test_top10_df.count()

#### 합성데이터 (X_syn_train, X_syn_test, y_syn_train, y_syn_test)

- 합성데이터 회귀모델 학습 

In [38]:
lr_syn = LinearRegression()
lr_syn.fit(X_syn_train, y_syn_train)

LinearRegression()

In [39]:
score = lr_syn.score(X_syn_test, y_syn_test)
print("Model score:", score)

Model score: -0.0281892365941947


- 합성데이터 예측값

In [40]:
y_syn_pred = lr_syn.predict(X_syn_test)

In [41]:
index = X_syn_test['dec_num'].index.tolist()

In [42]:
df = pd.DataFrame({'index' : index ,'pred': y_syn_pred})

- 합성데이터 우범 회귀 예측값 상위 5%의 우범률

In [43]:
top_5percent = df['pred'].quantile(0.95)
result = df[df['pred'] > top_5percent]

In [44]:
y_test_top5 = y_syn_test[y_syn_test.index.isin(result['index'])]

In [45]:
y_test_top5_df = pd.DataFrame({'crime_yn' :y_test_top5})

In [46]:
syn_5 = y_test_top5_df[y_test_top5_df['crime_yn']==1].count() / y_test_top5_df.count()

- 합성데이터 우범 회귀 예측값 상위 10%의 우범률

In [47]:
top_10percent = df['pred'].quantile(0.90)
result = df[df['pred'] > top_10percent]

In [48]:
y_test_top10 = y_syn_test[y_syn_test.index.isin(result['index'])]

In [49]:
y_test_top10_df = pd.DataFrame({'crime_yn' : y_test_top10})

In [50]:
syn_10= y_test_top10_df[y_test_top10_df['crime_yn']==1].count() / y_test_top10_df.count()

#### 우범률 정리 - 선형 회귀분석

In [51]:
df_result_lr = pd.DataFrame({'category': 'LinearRegression','base_5' : base_5, 'base_10': base_10,'syn_5': syn_5,'syn_10': syn_10})

In [52]:
df_result_lr

,category,base_5,base_10,syn_5,syn_10
crime_yn,LinearRegression,0.35,0.309259,0.333333,0.3


### 랜덤포레스트

In [53]:
from sklearn.ensemble import RandomForestRegressor

#### 원본데이터 (X_train, X_test, y_train, y_test)

- 원본데이터 랜덤포레스트 회귀모델 학습 

In [54]:
rf = RandomForestRegressor(n_estimators=50, random_state=42)

In [55]:
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=50, random_state=42)

- 원본데이터 예측값

In [56]:
y_pred = rf.predict(X_test)

In [57]:
index = X_test['dec_num'].index.tolist()

In [58]:
df = pd.DataFrame({'index' : index ,'pred': y_pred})

- 원본데이터 우범 회귀 예측값 상위 5%의 우범률

In [59]:
top_5percent = df['pred'].quantile(0.95)
result = df[df['pred'] > top_5percent]

In [60]:
y_test_top5 = y_test[y_test.index.isin(result['index'])]

In [61]:
y_test_top5_df = pd.DataFrame({'crime_yn' :y_test_top5})

In [62]:
base_5 = y_test_top5_df[y_test_top5_df['crime_yn']==1].count() / y_test_top5_df.count()

- 원본데이터 우범 회귀 예측값 상위 10%의 우범률

In [63]:
top_10percent = df['pred'].quantile(0.90)
result = df[df['pred'] > top_10percent]

In [64]:
y_test_top10 = y_test[y_test.index.isin(result['index'])]

In [65]:
y_test_top10_df = pd.DataFrame({'crime_yn' : y_test_top10})

In [66]:
base_10 = y_test_top10_df[y_test_top10_df['crime_yn']==1].count() / y_test_top10_df.count()

#### 합성데이터 (X_syn_train, X_syn_test, y_syn_train, y_syn_test)

- 합성데이터 랜덤포레스트 회귀모델 학습 

In [67]:
rf_syn = RandomForestRegressor(n_estimators=50, random_state=42)

In [68]:
rf_syn.fit(X_syn_train, y_syn_train)

RandomForestRegressor(n_estimators=50, random_state=42)

- 합성데이터 예측값

In [69]:
y_syn_pred = rf_syn.predict(X_syn_test)

In [70]:
index = X_syn_test['dec_num'].index.tolist()

In [71]:
df = pd.DataFrame({'index' : index ,'pred': y_syn_pred})

- 합성데이터 우범 회귀 예측값 상위 5%의 우범률

In [72]:
top_5percent = df['pred'].quantile(0.95)
result = df[df['pred'] > top_5percent]

In [73]:
y_test_top5 = y_syn_test[y_syn_test.index.isin(result['index'])]

In [74]:
y_test_top5_df = pd.DataFrame({'crime_yn' :y_test_top5})

In [75]:
syn_5 = y_test_top5_df[y_test_top5_df['crime_yn']==1].count() / y_test_top5_df.count()

- 합성데이터 우범 회귀 예측값 상위 10%의 우범률

In [76]:
top_10percent = df['pred'].quantile(0.90)
result = df[df['pred'] > top_10percent]

In [77]:
y_test_top10 = y_syn_test[y_syn_test.index.isin(result['index'])]

In [78]:
y_test_top10_df = pd.DataFrame({'crime_yn' : y_test_top10})

In [79]:
syn_10 = y_test_top10_df[y_test_top10_df['crime_yn']==1].count() / y_test_top10_df.count()

#### 우범률 정리 - 랜덤포레스트 

In [80]:
df_result_rf = pd.DataFrame({'category': 'RandomForestRegressor','base_5' : base_5, 'base_10': base_10,'syn_5': syn_5,'syn_10': syn_10})

In [81]:
df_result_rf

,category,base_5,base_10,syn_5,syn_10
crime_yn,RandomForestRegressor,0.89759,0.824742,0.714286,0.615385


### xgBoost

In [82]:
import xgboost as xgb

#### 원본데이터 (X_train, X_test, y_train, y_test)

- 원본데이터 xgBoost 회귀모델 학습 

In [83]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [84]:
params = {
    "max_depth": 2,
    "eta": 0.1,
    "subsample": 0.5,
    "colsample_bytree": 0.5,
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
}
num_round = 100

In [85]:
xgb_model = xgb.train(params, dtrain, num_round)

- 원본데이터 예측값

In [86]:
y_pred = xgb_model.predict(dtest)

In [87]:
index = X_test['dec_num'].index.tolist()

In [88]:
df = pd.DataFrame({'index' : index ,'pred': y_pred})

- 원본데이터 우범 회귀 예측값 상위 5%의 우범률

In [89]:
top_5percent = df['pred'].quantile(0.95)
result = df[df['pred'] > top_5percent]

In [90]:
y_test_top5 = y_test[y_test.index.isin(result['index'])]

In [91]:
y_test_top5_df = pd.DataFrame({'crime_yn' :y_test_top5})

In [92]:
base_5 = y_test_top5_df[y_test_top5_df['crime_yn']==1].count() / y_test_top5_df.count()

- 원본데이터 우범 회귀 예측값 상위 10%의 우범률

In [93]:
top_10percent = df['pred'].quantile(0.90)
result = df[df['pred'] > top_10percent]

In [94]:
y_test_top10 = y_test[y_test.index.isin(result['index'])]

In [95]:
y_test_top10_df = pd.DataFrame({'crime_yn' : y_test_top10})

In [96]:
base_10 = y_test_top10_df[y_test_top10_df['crime_yn']==1].count() / y_test_top10_df.count()

#### 합성데이터 (X_syn_train, X_syn_test, y_syn_train, y_syn_test)

- 합성데이터 xgBoost 회귀모델 학습 

In [97]:
dtrain_syn = xgb.DMatrix(X_syn_train, label=y_syn_train)
dtest_syn = xgb.DMatrix(X_syn_test, label=y_syn_test)

In [98]:
params = {
    "max_depth": 2,
    "eta": 0.1,
    "subsample": 0.5,
    "colsample_bytree": 0.5,
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
}
num_round = 100

In [99]:
xgb_model_syn = xgb.train(params, dtrain_syn, num_round)

- 합성데이터 예측값

In [100]:
y_syn_pred = xgb_model_syn.predict(dtest_syn)

In [101]:
index = X_syn_test['dec_num'].index.tolist()

In [102]:
df = pd.DataFrame({'index' : index ,'pred': y_syn_pred})

- 합성데이터 우범 회귀 예측값 상위 5%의 우범률

In [103]:
top_5percent = df['pred'].quantile(0.95)
result = df[df['pred'] > top_5percent]

In [104]:
y_test_top5 = y_syn_test[y_syn_test.index.isin(result['index'])]

In [105]:
y_test_top5_df = pd.DataFrame({'crime_yn' :y_test_top5})

In [106]:
syn_5 = y_test_top5_df[y_test_top5_df['crime_yn']==1].count() / y_test_top5_df.count()

- 합성데이터 우범 회귀 예측값 상위 10%의 우범률

In [107]:
top_10percent = df['pred'].quantile(0.90)
result = df[df['pred'] > top_10percent]

In [108]:
y_test_top10 = y_syn_test[y_syn_test.index.isin(result['index'])]

In [109]:
y_test_top10_df = pd.DataFrame({'crime_yn' : y_test_top10})

In [110]:
syn_10 = y_test_top10_df[y_test_top10_df['crime_yn']==1].count() / y_test_top10_df.count()

#### 우범률 정리 - xgBoost 

In [111]:
df_result_xgb = pd.DataFrame({'category': 'xgboost','base_5' : base_5, 'base_10': base_10,'syn_5': syn_5,'syn_10': syn_10})

In [112]:
df_result_xgb

,category,base_5,base_10,syn_5,syn_10
crime_yn,xgboost,0.414815,0.363889,0.466667,0.4


## 모델별 우범률 정리

In [114]:
total_result = pd.concat([df_result_lr, df_result_rf, df_result_xgb])

In [115]:
total_result = total_result.reset_index(drop=True)

In [116]:
total_result

,category,base_5,base_10,syn_5,syn_10
0,LinearRegression,0.350000,0.309259,0.333333,0.300000
1,RandomForestRegressor,0.897590,0.824742,0.714286,0.615385
2,xgboost,0.414815,0.363889,0.466667,0.400000


In [1]:
!nvidia-smi

Mon Mar 20 14:41:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 528.24       Driver Version: 528.24       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   48C    P8     7W /  80W |    749MiB /  6144MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------